In [1]:
from ortools.sat.python import cp_model

import csv

import pandas as pd

import random



# Fixing seed for random

random.seed(40)



TotalPPE = 700

Hospitals = []

NumHospitals = 5

PatientsPerHospital = [30, 20, 35, 34, 15]

StaffPerHospital = [50, 55, 65, 72, 45]

SeverityLevels = 5

# Number of pacients per level of severity

PacientsPerSeverity = {}

# Generating the number of pacients of each severity level

for i in range(NumHospitals):

	Hospitals.append(i)

	cumulated = 0

	for s in range(SeverityLevels):

		if(s < 4):

			random_number = random.randint(0, PatientsPerHospital[i] - cumulated)

		else:

			random_number = PatientsPerHospital[i] - cumulated

		cumulated = cumulated + random_number

		PacientsPerSeverity[(i,s + 1)] = random_number



# Defining model

model = cp_model.CpModel()



# Integer Variable: total number of PPE used for patients at each hospital

PPEPacientsHospital = [model.NewIntVar(0, TotalPPE, 'PPEPacientsHospital%i' % i) for i in range(NumHospitals)]

# Integer Variable: total number of PPE used for patients in severe condition (5) at each hospital

PPEPacientsSevere = [model.NewIntVar(0, TotalPPE, 'PPEPacientsSevere%i' % i) for i in range(NumHospitals)]

# Integer Variable: total number of PPE at each hospital - minimum will be the number of staff at this hospital

PPEHospital = [model.NewIntVar(StaffPerHospital[i], TotalPPE, 'PPEStaffHospital%i' % i) for i in range(NumHospitals)]



# All PPE should be used

model.Add(sum(PPEHospital[h] for h in range(NumHospitals)) == TotalPPE)



# Maximize Number of PPE per hospital based on the number of patients in severe condition and staff

model.Maximize(sum(PacientsPerSeverity[h,5]*StaffPerHospital[i]*PPEHospital[h] for h in range(NumHospitals)))



# Print solution

solver = cp_model.CpSolver()

status = solver.Solve(model)

for h in range(NumHospitals):

	print(h, solver.Value(PPEHospital[h]))







0 50
1 55
2 478
3 72
4 45
